<a href="https://colab.research.google.com/github/AlchemyGeek/Coursera-Tensorflow/blob/master/CNN-Week4-SignLangClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import csv
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import files

The data for this exercise is available at: https://www.kaggle.com/datamunge/sign-language-mnist/home

Sign up and download to find 2 CSV files: sign_mnist_test.csv and sign_mnist_train.csv -- You will upload both of them using this button before you can continue.


In [10]:
uploaded=files.upload()

Saving sign_mnist_test.csv to sign_mnist_test (1).csv
Saving sign_mnist_train.csv to sign_mnist_train (1).csv


KeyboardInterrupt: ignored

In [33]:
def get_data(filename):
  # You will need to write code that will read the file passed
  # into this function. The first line contains the column headers
  # so you should ignore it
  # Each successive line contians 785 comma separated values between 0 and 255
  # The first value is the label
  # The rest are the pixel values for that picture
  # The function will return 2 np.array types. One with all the labels
  # One with all the images
  #
  # Tips: 
  # If you read a full line (as 'row') then row[0] has the label
  # and row[1:785] has the 784 pixel values
  # Take a look at np.array_split to turn the 784 pixels into 28x28
  # You are reading in strings, but need the values to be floats
  # Check out np.array().astype for a conversion
    with open(filename,"r") as training_file:
      print(filename)

      #Calculate lines
      count = 0
      while training_file.readline(  ):
        count = count + 1
      
      training_file.seek(0)
      print(count)

      #create arrays
      images = np.zeros([count,28,28])
      labels = np.zeros([count])

      print(images.shape)

      #ignore labels
      training_file.readline()
    
      #for each line in file
      data = training_file.readline()
      print(data)
      count = 0
      while data:



        # Extra columns 
        col= data.split(",")

        print(count,len(col),training_file.tell())
      
        # Retrieve label
        labels[count] = float(col[0])

        # Retrieve image
        col = col[1:]
        # TODO: Convert to Float
        images[count] = np.array_split(col,28)


        # Next line
        count = count + 1
        data = training_file.readline()

    return images, labels

testing_images, testing_labels = get_data('sign_mnist_test.csv')
training_images, training_labels = get_data('sign_mnist_train.csv')
testing_images, testing_labels = get_data('sign_mnist_test.csv')

# Keep these
print(training_images.shape)
print(training_labels.shape)
print(testing_images.shape)
print(testing_labels.shape)

# Their output should be:
# (27455, 28, 28)
# (27455,)
# (7172, 28, 28)
# (7172,)

Streaming output truncated to the last 5000 lines.
15086 785 45773270
15087 785 45776397
15088 785 45779481
15089 785 45782454
15090 785 45785571
15091 785 45788608
15092 785 45791632
15093 785 45794771
15094 785 45797489
15095 785 45800518
15096 785 45803598
15097 785 45806699
15098 785 45809664
15099 785 45812744
15100 785 45815845
15101 785 45818753
15102 785 45821774
15103 785 45824805
15104 785 45827884
15105 785 45830869
15106 785 45834000
15107 785 45837111
15108 785 45840162
15109 785 45843071
15110 785 45845861
15111 785 45848979
15112 785 45851949
15113 785 45854953
15114 785 45858039
15115 785 45861134
15116 785 45864229
15117 785 45867261
15118 785 45870254
15119 785 45873370
15120 785 45876441
15121 785 45879442
15122 785 45882434
15123 785 45885487
15124 785 45888547
15125 785 45891145
15126 785 45894122
15127 785 45897118
15128 785 45900184
15129 785 45902888
15130 785 45905896
15131 785 45909017
15132 785 45912070
15133 785 45915105
15134 785 45918242
15135 785 45921232

ValueError: ignored

# New Section

In [0]:
# In this section you will have to add another dimension to the data
# So, for example, if your array is (10000, 28, 28)
# You will need to make it (10000, 28, 28, 1)
# Hint: np.expand_dims

training_images = # Your Code Here
testing_images = # Your Code Here

# Create an ImageDataGenerator and do Image Augmentation
train_datagen = ImageDataGenerator(
    # Your Code Here
    )

validation_datagen = ImageDataGenerator(
    # Your Code Here)
    
# Keep These
print(training_images.shape)
print(testing_images.shape)
    
# Their output should be:
# (27455, 28, 28, 1)
# (7172, 28, 28, 1)

In [0]:
# Define the model
# Use no more than 2 Conv2D and 2 MaxPooling2D
model = tf.keras.models.Sequential([
    # Your Code Here
    )

# Compile Model. 
model.compile(# Your Code Here)

# Train the Model
history = model.fit_generator(# Your Code Here)

model.evaluate(testing_images, testing_labels)
    
# The output from model.evaluate should be close to:
[6.92426086682151, 0.56609035]


In [0]:
# Plot the chart for accuracy and loss on both training and validation

import matplotlib.pyplot as plt
acc = # Your Code Here
val_acc = # Your Code Here
loss = # Your Code Here
val_loss = # Your Code Here

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()